* Day Count Convention	30/360
* O/N Leg Frequency	Daily
* Fixed Leg Frequency	Annual


In [48]:
import pandas as pd
import matplotlib.pyplot as plt

# handy it seems
# https://docs.sympy.org/latest/modules/solvers/solvers.html
from sympy.solvers import solve
from sympy import Symbol


# read data
ois_data = pd.read_csv("../data/OIS_Data.csv")
irs_data = pd.read_csv("../data/IRS_Data.csv")
ois_data.columns = map(str.lower, ois_data.columns)
irs_data.columns = map(str.lower, irs_data.columns)

# use dict comprehension?
tenor_mapping = {
    "6m": 0.5,
    "1y": 1,
    "2y": 2,
    "3y": 3,
    "4y": 4,
    "5y": 5,
    "7y": 7,
    "10y": 10,
    "15y": 15,
    "20y": 20,
    "30y": 30,
}

# OIS processing
ois_data["tenor"] = ois_data["tenor"].map(tenor_mapping)
ois_data["rate"] = ois_data["rate"].str.strip("%").astype(float) / 100.0

# DAY COUNT CONVENTION IS 30/360
FULL_YEAR = 360

In [49]:
ois_data

,tenor,product,rate
0,0.5,OIS,0.00250
1,1.0,OIS,0.00300
2,2.0,OIS,0.00325
3,3.0,OIS,0.00335
4,4.0,OIS,0.00350
5,5.0,OIS,0.00360
6,7.0,OIS,0.00400
7,10.0,OIS,0.00450
8,15.0,OIS,0.00500
9,20.0,OIS,0.00525


##  solve for < 1 y stuff ##

Because these 2 are anchors i believe

In [50]:
f_6m = Symbol("f_6m")
f_6m = solve((1 + 0.5 * ois_data.loc[0, "rate"]) ** (1 / 180) - (1 + f_6m / 360))[0]
print(f_6m)

0.00249844747059136


In [51]:
f_1y = Symbol("f_1y")
f_1y = solve(
    ((1 + 1 * ois_data.loc[1, "rate"]) / (1 + f_6m / 360) ** 180) ** (1 / 180)
    - (1 + f_1y / 360)
)[0]
print(f_1y)

0.00349259610064934


In [52]:
ois_data.loc[0, "disc_factor"] = 1 / (1 + f_6m / 360) ** 180
ois_data.loc[1, "disc_factor"] = 1 / (
    ((1 + f_6m / 360) ** 180) * ((1 + f_1y / 360) ** 180)
)

In [53]:
ois_data

,tenor,product,rate,disc_factor
0,0.5,OIS,0.00250,0.998751560549318
1,1.0,OIS,0.00300,0.997008973080741
2,2.0,OIS,0.00325,NaN
3,3.0,OIS,0.00335,NaN
4,4.0,OIS,0.00350,NaN
5,5.0,OIS,0.00360,NaN
6,7.0,OIS,0.00400,NaN
7,10.0,OIS,0.00450,NaN
8,15.0,OIS,0.00500,NaN
9,20.0,OIS,0.00525,NaN


## Let's try the collapsing shit, thx eko ##
I can test up until 5 first

In [54]:
y_1_funny = (
    ((1 + f_6m / 360) ** 180)
    * ((1 + f_1y / 360) ** 180)
    * ois_data.loc[1, "disc_factor"]
)
# youd be surprised
print(y_1_funny)

1.00000000000000


In [55]:
for k in range(2, 6):
    denominator = 1 / (1 + 1 * ois_data.loc[k, "rate"])
    sum_prev_disc_rate = ois_data.loc[1:k, "disc_factor"].sum()
    print(sum_prev_disc_rate)
    numerator = y_1_funny - 1 * ois_data.loc[k, "rate"] * sum_prev_disc_rate
    ois_data.loc[k, "disc_factor"] = numerator * denominator

0.997008973080741
1.99053971899401
2.98055486021230
3.96667150992755


In [56]:
ois_data

,tenor,product,rate,disc_factor
0,0.5,OIS,0.00250,0.998751560549318
1,1.0,OIS,0.00300,0.997008973080741
2,2.0,OIS,0.00325,0.993530745913270
3,3.0,OIS,0.00335,0.990015141218289
4,4.0,OIS,0.00350,0.986116649715253
5,5.0,OIS,0.00360,0.982184119733221
6,7.0,OIS,0.00400,NaN
7,10.0,OIS,0.00450,NaN
8,15.0,OIS,0.00500,NaN
9,20.0,OIS,0.00525,NaN


## i will raw dog 7,10,15,20,30 because i can and im not really smart, DEFINITELY GOT SOME WAY TO DP THIS ##

In [57]:
# 7.0
k = 6
denominator = 1 / (1 + 1 * 1.5 * ois_data.loc[k, "rate"])
sum_prev_disc_rate = ois_data.loc[1:k, "disc_factor"].sum()
sum_prev_disc_rate += 0.5 * ois_data.loc[k - 1, "disc_factor"]
numerator = y_1_funny - 1 * ois_data.loc[k, "rate"] * sum_prev_disc_rate
ois_data.loc[k, "disc_factor"] = numerator * denominator

In [58]:
# 15.0
k = 7
denominator = 1 / (1 + 1 * (1 + 1 / 3 + 2 / 3) * ois_data.loc[k, "rate"])
sum_prev_disc_rate = ois_data.loc[1:6, "disc_factor"].sum()
sum_prev_disc_rate += 0.5 * ois_data.loc[5, "disc_factor"]  # add the d6 interp
sum_prev_disc_rate += 0.5 * ois_data.loc[6, "disc_factor"]  # add the d6 interp
sum_prev_disc_rate += 1 * ois_data.loc[6, "disc_factor"]  # add d7
sum_prev_disc_rate += (1 / 3) * ois_data.loc[6, "disc_factor"]  # add the d8,9 interp
sum_prev_disc_rate += (2 / 3) * ois_data.loc[6, "disc_factor"]  # add the d8,9 interp

numerator = y_1_funny - 1 * ois_data.loc[k, "rate"] * sum_prev_disc_rate
ois_data.loc[k, "disc_factor"] = numerator * denominator

In [59]:
ois_data

,tenor,product,rate,disc_factor
0,0.5,OIS,0.00250,0.998751560549318
1,1.0,OIS,0.00300,0.997008973080741
2,2.0,OIS,0.00325,0.993530745913270
3,3.0,OIS,0.00335,0.990015141218289
4,4.0,OIS,0.00350,0.986116649715253
5,5.0,OIS,0.00360,0.982184119733221
6,7.0,OIS,0.00400,0.972405774594325
7,10.0,OIS,0.00450,0.951640083694020
8,15.0,OIS,0.00500,NaN
9,20.0,OIS,0.00525,NaN


In [60]:
# 15.0
k = 8
denominator = 1 / (1 + 1 * (1 + (1 + 2 + 3 + 4) / 5) * ois_data.loc[k, "rate"])
sum_prev_disc_rate = ois_data.loc[1:6, "disc_factor"].sum()
sum_prev_disc_rate += 0.5 * ois_data.loc[5, "disc_factor"]  # add the d6 interp
sum_prev_disc_rate += 0.5 * ois_data.loc[6, "disc_factor"]  # add the d6 interp
sum_prev_disc_rate += 1 * ois_data.loc[6, "disc_factor"]  # add d7
sum_prev_disc_rate += (1 / 3) * ois_data.loc[6, "disc_factor"]  # add the d8,9 interp
sum_prev_disc_rate += (2 / 3) * ois_data.loc[6, "disc_factor"]  # add the d8,9 interp
sum_prev_disc_rate += (1 / 3) * ois_data.loc[7, "disc_factor"]  # add the d8,9 interp
sum_prev_disc_rate += (2 / 3) * ois_data.loc[7, "disc_factor"]  # add the d8,9 interp
sum_prev_disc_rate += 1 * ois_data.loc[7, "disc_factor"]  # add d10
sum_prev_disc_rate += (1 / 5) * ois_data.loc[7, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (2 / 5) * ois_data.loc[7, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (3 / 5) * ois_data.loc[7, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (4 / 5) * ois_data.loc[7, "disc_factor"]  # add the d11-14 interp

numerator = y_1_funny - 1 * ois_data.loc[k, "rate"] * sum_prev_disc_rate
ois_data.loc[k, "disc_factor"] = numerator * denominator

In [61]:
ois_data

,tenor,product,rate,disc_factor
0,0.5,OIS,0.00250,0.998751560549318
1,1.0,OIS,0.00300,0.997008973080741
2,2.0,OIS,0.00325,0.993530745913270
3,3.0,OIS,0.00335,0.990015141218289
4,4.0,OIS,0.00350,0.986116649715253
5,5.0,OIS,0.00360,0.982184119733221
6,7.0,OIS,0.00400,0.972405774594325
7,10.0,OIS,0.00450,0.951640083694020
8,15.0,OIS,0.00500,0.922906757461165
9,20.0,OIS,0.00525,NaN


In [62]:
# 20.0
k = 9
denominator = 1 / (1 + 1 * (1 + (1 + 2 + 3 + 4) / 5) * ois_data.loc[k, "rate"])
sum_prev_disc_rate = ois_data.loc[1:6, "disc_factor"].sum()
sum_prev_disc_rate += 0.5 * ois_data.loc[5, "disc_factor"]  # add the d6 interp
sum_prev_disc_rate += 0.5 * ois_data.loc[6, "disc_factor"]  # add the d6 interp
sum_prev_disc_rate += 1 * ois_data.loc[6, "disc_factor"]  # add d7
sum_prev_disc_rate += (1 / 3) * ois_data.loc[6, "disc_factor"]  # add the d8,9 interp
sum_prev_disc_rate += (2 / 3) * ois_data.loc[6, "disc_factor"]  # add the d8,9 interp
sum_prev_disc_rate += (1 / 3) * ois_data.loc[7, "disc_factor"]  # add the d8,9 interp
sum_prev_disc_rate += (2 / 3) * ois_data.loc[7, "disc_factor"]  # add the d8,9 interp
sum_prev_disc_rate += 1 * ois_data.loc[7, "disc_factor"]  # add d10
sum_prev_disc_rate += (1 / 5) * ois_data.loc[7, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (2 / 5) * ois_data.loc[7, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (3 / 5) * ois_data.loc[7, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (4 / 5) * ois_data.loc[7, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (1 / 5) * ois_data.loc[8, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (2 / 5) * ois_data.loc[8, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (3 / 5) * ois_data.loc[8, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (4 / 5) * ois_data.loc[8, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += 1 * ois_data.loc[8, "disc_factor"]  # add d15
sum_prev_disc_rate += (1 / 5) * ois_data.loc[8, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (2 / 5) * ois_data.loc[8, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (3 / 5) * ois_data.loc[8, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (4 / 5) * ois_data.loc[8, "disc_factor"]  # add the d11-14 interp

numerator = y_1_funny - 1 * ois_data.loc[k, "rate"] * sum_prev_disc_rate
ois_data.loc[k, "disc_factor"] = numerator * denominator

In [63]:
# 30.0
k = 10
denominator = 1 / (
    1 + 1 * (1 + (1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9) / 10) * ois_data.loc[k, "rate"]
)
sum_prev_disc_rate = ois_data.loc[1:6, "disc_factor"].sum()
sum_prev_disc_rate += 0.5 * ois_data.loc[5, "disc_factor"]  # add the d6 interp
sum_prev_disc_rate += 0.5 * ois_data.loc[6, "disc_factor"]  # add the d6 interp
sum_prev_disc_rate += 1 * ois_data.loc[6, "disc_factor"]  # add d7
sum_prev_disc_rate += (1 / 3) * ois_data.loc[6, "disc_factor"]  # add the d8,9 interp
sum_prev_disc_rate += (2 / 3) * ois_data.loc[6, "disc_factor"]  # add the d8,9 interp
sum_prev_disc_rate += (1 / 3) * ois_data.loc[7, "disc_factor"]  # add the d8,9 interp
sum_prev_disc_rate += (2 / 3) * ois_data.loc[7, "disc_factor"]  # add the d8,9 interp
sum_prev_disc_rate += 1 * ois_data.loc[7, "disc_factor"]  # add d10
sum_prev_disc_rate += (1 / 5) * ois_data.loc[7, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (2 / 5) * ois_data.loc[7, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (3 / 5) * ois_data.loc[7, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (4 / 5) * ois_data.loc[7, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (1 / 5) * ois_data.loc[8, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (2 / 5) * ois_data.loc[8, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (3 / 5) * ois_data.loc[8, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += (4 / 5) * ois_data.loc[8, "disc_factor"]  # add the d11-14 interp
sum_prev_disc_rate += 1 * ois_data.loc[8, "disc_factor"]  # add d15
sum_prev_disc_rate += (1 / 5) * ois_data.loc[8, "disc_factor"]  # add the d16-19 interp
sum_prev_disc_rate += (2 / 5) * ois_data.loc[8, "disc_factor"]  # add the d16-19 interp
sum_prev_disc_rate += (3 / 5) * ois_data.loc[8, "disc_factor"]  # add the d16-19 interp
sum_prev_disc_rate += (4 / 5) * ois_data.loc[8, "disc_factor"]  # add the d16-19 interp
sum_prev_disc_rate += (1 / 5) * ois_data.loc[9, "disc_factor"]  # add the d16-19 interp
sum_prev_disc_rate += (2 / 5) * ois_data.loc[9, "disc_factor"]  # add the d16-19 interp
sum_prev_disc_rate += (3 / 5) * ois_data.loc[9, "disc_factor"]  # add the d16-19 interp
sum_prev_disc_rate += (4 / 5) * ois_data.loc[9, "disc_factor"]  # add the d16-19 interp
sum_prev_disc_rate += 1 * ois_data.loc[9, "disc_factor"]  # add d20
sum_prev_disc_rate += (1 / 10) * ois_data.loc[9, "disc_factor"]  # add the d21-29 interp
sum_prev_disc_rate += (2 / 10) * ois_data.loc[9, "disc_factor"]  # add the d21-29 interp
sum_prev_disc_rate += (3 / 10) * ois_data.loc[9, "disc_factor"]  # add the d21-29 interp
sum_prev_disc_rate += (4 / 10) * ois_data.loc[9, "disc_factor"]  # add the d21-29 interp
sum_prev_disc_rate += (5 / 10) * ois_data.loc[9, "disc_factor"]  # add the d21-29 interp
sum_prev_disc_rate += (6 / 10) * ois_data.loc[9, "disc_factor"]  # add the d21-29 interp
sum_prev_disc_rate += (7 / 10) * ois_data.loc[9, "disc_factor"]  # add the d21-29 interp
sum_prev_disc_rate += (8 / 10) * ois_data.loc[9, "disc_factor"]  # add the d21-29 interp
sum_prev_disc_rate += (9 / 10) * ois_data.loc[9, "disc_factor"]  # add the d21-29 interp

numerator = y_1_funny - 1 * ois_data.loc[k, "rate"] * sum_prev_disc_rate
ois_data.loc[k, "disc_factor"] = numerator * denominator

In [64]:
ois_data

,tenor,product,rate,disc_factor
0,0.5,OIS,0.00250,0.998751560549318
1,1.0,OIS,0.00300,0.997008973080741
2,2.0,OIS,0.00325,0.993530745913270
3,3.0,OIS,0.00335,0.990015141218289
4,4.0,OIS,0.00350,0.986116649715253
5,5.0,OIS,0.00360,0.982184119733221
6,7.0,OIS,0.00400,0.972405774594325
7,10.0,OIS,0.00450,0.951640083694020
8,15.0,OIS,0.00500,0.922906757461165
9,20.0,OIS,0.00525,0.895261210318366
